In [ ]:
import os
from tracktool import load_graph, get_traccuracy_graph
from traccuracy.loaders import load_ctc_data
from traccuracy.matchers import CTCMatched
from traccuracy.metrics import CTCMetrics

In [ ]:
DATA_ROOT = '/home/draga/PhD/data/cell_tracking_challenge/ST_Segmentations/'
DS_NAME = 'Fluo-N2DL-HeLa/'
SEQ = '02_ST'
GT_PATH = os.path.join(DATA_ROOT, DS_NAME, '02_GT/TRA/')
SEG_PATH = os.path.join(DATA_ROOT, DS_NAME, SEQ, 'SEG/')

In [ ]:
seg_ims, flow_graph = load_graph(SEG_PATH)
flow_graph.solve()

In [ ]:
tg = get_traccuracy_graph(flow_graph, seg_ims)
gt_graph = load_ctc_data(GT_PATH)
match = CTCMatched(gt_graph, tg)
raw_ctc = CTCMetrics(match)
res = raw_ctc.results
